In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
test_set = pd.read_csv("../data/ml-1m/testing_set_seed123_ml1m.csv",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set


In [ ]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

In [ ]:


mf = "model_reco/1m/mf/reco_matrix_mf_1m_1000.npy"
wmf = "model_reco/1m/wmf/reco_matrix_wmf_1m_1000.npy"
neumf = "model_reco/1m/neumf/reco_matrix_neumf_1m_1000.npy"
vaecf = "model_reco/1m/vaecf/reco_matrix_vaecg_1m_1000.npy"
reco_matrix = np.load(f"../{mf}")
reco_matrix.shape

In [ ]:

from cornac.metrics import NDCG, HitRatio

hr=HitRatio(20)
alist=[]
o=0

sum_hr=0
for u in range(reco_matrix[o].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[o][u][:20])
    sum_hr=sum_hr+x
plain_hr=sum_hr/reco_matrix[o].shape[0]

plain_hr


In [ ]:
# from NDCG import NDCG
ndcg=NDCG(20)
alist=[]
sum_ndcg=0
o = 0
for u in range(reco_matrix[0].shape[0]):
    x=ndcg.compute(all_users_tp[u],reco_matrix[o][u][:20])
    
    # print(f"x {x} tp {all_users_tp[u]} reco {reco_matrix[u][:20]}")
    # alist.append(x)
    sum_ndcg=sum_ndcg+x
    # print(x)
plain_ndcg=sum_ndcg/reco_matrix[o].shape[0]

plain_ndcg
# 0.06877010356676778

In [12]:
all_pds =[]
reco_matrix_fair=[]
sens="occupation"
model = "mf"
# here we load the recommendations over different vaues of beta
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_1.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_2.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_3.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_4.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_5.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_6.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_7.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_8.csv",sep=",",))
for j in range(len(all_pds)):
    reco_m =[]
    for i in range(all_pds[j].__len__()):
        reco_m.append(np.array(all_pds[j].iloc[i][1:]))
    reco_matrix_fair.append(reco_m)
    



In [ ]:

hr=HitRatio(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_hr]+hrlist

In [ ]:


hr=NDCG(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_ndcg]+hrlist


In [ ]:
movie = "../data/ml-1m/i_id_mapping_genre.csv"

df_movie = pd.read_csv(movie, sep="\t")
print(f"shape: {df_movie.shape}")


df_movie.columns = ["item_id", "Name", "genres", "itemID"]
df_movie[:4]
movies = df_movie.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0

for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1
movies
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
movies

movies

In [ ]:
users = pd.read_csv("../data/ml-1m/u_id_mapping_demographic.csv", sep="\t")
# users


users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
# user_features_numpy = users.to_numpy()
users

In [ ]:
import os
os.chdir("/Users/name/Desktop/work/reranking_fairnes")

from metrics.utils import map_age
users["Age_Code"] = users["Age"].apply(map_age)
users

In [19]:
from mymetrics.GenrePrecisionMulti import GenrePrecisionMulti
from mymetrics.GenreNDCGMulti import GenreNDCGMulti
top_k = 20
###intialize them
gp = GenrePrecisionMulti(users, unique_genres, top_k)
gn = GenreNDCGMulti(users, unique_genres, top_k)



In [ ]:
reco_matrix[0].shape
new_rmatrix = np.zeros(( reco_matrix.shape[0], 20), dtype=int)
for i in range(reco_matrix.shape[0]):
    new_rmatrix[i]=reco_matrix[i][:20]
new_rmatrix.shape


In [ ]:
sens_attr = "Occupation"
gp_plain = gp.compute(new_rmatrix, movies, sens_attr)
gp_plain

In [ ]:
gn_plain = gn.compute(new_rmatrix, movies, sens_attr)
gn_plain

In [ ]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gp.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    print(f"{x[0]} is the bias score for b={i}")


In [ ]:
[gp_plain[0]]+xlist

In [ ]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gn.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    print(f"{x[0]} is the bias score for b={i}")


In [ ]:
[gn_plain[0]]+xlist